# Session 5 (2/9): Classical Filtering for Data Assimilation (Non-ML)

This notebook focuses on classical filtering algorithms in sequential data assimilation.

Outline:
- Data/model setup
- Particle filters
- Gaussian affine-map filters
- Ensemble implementations


## 1. Unified Notation and Two Canonical Testbeds

We use the standard hidden-state model
$$
\begin{aligned}
V_{j+1}^{\dagger} &= \Psi(V_j^{\dagger}) + \xi_j^{\dagger}, \quad
\xi_j^{\dagger} \sim \mathcal N(0,\Sigma), \\
y_{j+1}^{\dagger} &= h(V_{j+1}^{\dagger}) + \eta_{j+1}^{\dagger}, \quad
\eta_{j+1}^{\dagger} \sim \mathcal N(0,\Gamma).
\end{aligned}
$$
with observation history and filtering measures
$$
\begin{aligned}
&Y_j^{\dagger} = \{y_1^{\dagger},\dots,y_j^{\dagger}\}, \quad
\pi_j = \mathbb P(V_j^{\dagger}\mid Y_j^{\dagger}), \\
&\widehat\pi_{j+1} = \mathsf{P}\pi_j, \quad
\pi_{j+1} = \mathsf{A}_{j+1}\widehat\pi_{j+1}.
\end{aligned}
$$
and regularity assumptions $\Psi\in C^1(\mathbb R^d,\mathbb R^d),\ h\in C^1(\mathbb R^d,\mathbb R^k)$.


### 1.1 Test Dataset 1: Linear-Gaussian, d=2
$$
\begin{aligned}
v_{j+1} &= A v_j + \xi_j, \quad \xi_j \sim \mathcal N(0,\Sigma),\\
y_j &= H v_j + \eta_j, \quad \eta_j \sim \mathcal N(0,\Gamma).
\end{aligned}
$$
where
$$
A(\theta)=\begin{bmatrix}\cos\theta & -\sin\theta\\ \sin\theta & \cos\theta\end{bmatrix},
\qquad A^TA=I.
$$

Observation for this testbed is the first state component:
$$
h(v)=v_1,
\qquad H=\begin{bmatrix}1 & 0\end{bmatrix}.
$$

### 1.2 Test Dataset 2: Lorenz 63
$$
\begin{aligned}
\dot x &= \sigma(y-x), \\
\dot y &= x(\rho-z)-y, \\
\dot z &= xy-\beta z,
\end{aligned}
\qquad (\sigma,\rho,\beta)=\left(10,28,\frac{8}{3}\right).
$$
For Lorenz-63, we also observe the first state component only:
$$
h([x,y,z]^T)=x.
$$

### 1.3 DA Parameters and Numerical Details

#### Initial condition setup:

For the linear model, we take
$$
V_0 \sim \mathcal{N}(m_0, I),
$$
where the center can be a random initial point, e.g.
$$
m_0 \sim \mathcal{N}(0, I).
$$

For Lorenz-63, we use burn-in initialization: sample $m_0 \sim \mathcal{N}(0, I)$, run the model forward for many steps (100 steps here) to get $x_0$, and then set
$$
V_0 \sim \mathcal{N}(x_0, I).
$$


#### Noise levels
$$
\Sigma = \sigma_v^2 I,
\qquad \sigma_v = 0.01,
$$
$$
\Gamma = \sigma_y^2 I,
\qquad \sigma_y = 1.
$$

#### Numerical integral:
For Lorenz 63:
$$
\Delta t = 0.15 \quad \text{(assimilation time step)},
$$
$$
dt = 0.03 \quad \text{(integration time step, RK4 scheme)}.
$$


In [ ]:
try:
    import torch
except ImportError:
    torch = None
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

plt.style.use("seaborn-v0_8-whitegrid")
np.set_printoptions(precision=4, suppress=True)

TORCH_AVAILABLE = torch is not None
TORCH_GPU_AVAILABLE = bool(TORCH_AVAILABLE and torch.cuda.is_available())
TORCH_DEVICE = torch.device("cuda") if TORCH_GPU_AVAILABLE else (torch.device("cpu") if TORCH_AVAILABLE else None)
TORCH_DTYPE = torch.float64 if TORCH_AVAILABLE else None


def as_torch(x, device=None, dtype=None):
    if not TORCH_AVAILABLE:
        raise RuntimeError("PyTorch is not available in this environment.")
    if device is None:
        device = TORCH_DEVICE
    if dtype is None:
        dtype = TORCH_DTYPE
    return torch.as_tensor(x, device=device, dtype=dtype)


def symmetrize_torch(M):
    return 0.5 * (M + M.T)


def mvn_sample_torch(mean, cov, size=None, generator=None, eps=1e-9):
    mean = as_torch(mean)
    cov = as_torch(cov, device=mean.device, dtype=mean.dtype)
    cov = symmetrize_torch(cov)
    w, V = torch.linalg.eigh(cov)
    w = torch.clamp(w, min=eps)
    sqrt_cov = (V * torch.sqrt(w)) @ V.T
    d = mean.shape[0]
    if size is None:
        z = torch.randn(d, device=mean.device, dtype=mean.dtype, generator=generator)
        return mean + sqrt_cov @ z
    z = torch.randn((size, d), device=mean.device, dtype=mean.dtype, generator=generator)
    return z @ sqrt_cov.T + mean


def gaussian_logpdf_torch(x, mean, cov):
    x = as_torch(x).reshape(-1)
    mean = as_torch(mean, device=x.device, dtype=x.dtype).reshape(-1)
    cov = as_torch(cov, device=x.device, dtype=x.dtype)
    cov = symmetrize_torch(cov)
    d = x.shape[0]
    xc = x - mean
    sign, logdet = torch.linalg.slogdet(cov)
    if sign.item() <= 0:
        cov = cov + 1e-9 * torch.eye(cov.shape[0], device=cov.device, dtype=cov.dtype)
        _, logdet = torch.linalg.slogdet(cov)
    quad = xc @ torch.linalg.pinv(cov) @ xc
    const = torch.log(torch.tensor(2.0 * np.pi, device=x.device, dtype=x.dtype))
    return -0.5 * (d * const + logdet + quad)


def gaussian_logpdf_batch_torch(x, means, cov):
    means = as_torch(means)
    x = as_torch(x, device=means.device, dtype=means.dtype).reshape(1, -1)
    cov = as_torch(cov, device=means.device, dtype=means.dtype)
    cov = symmetrize_torch(cov)

    d = means.shape[1]
    xc = means - x
    sign, logdet = torch.linalg.slogdet(cov)
    if sign.item() <= 0:
        cov = cov + 1e-9 * torch.eye(cov.shape[0], device=cov.device, dtype=cov.dtype)
        _, logdet = torch.linalg.slogdet(cov)
    cov_inv = torch.linalg.pinv(cov)
    quad = torch.sum((xc @ cov_inv) * xc, dim=1)
    const = torch.log(torch.tensor(2.0 * np.pi, device=means.device, dtype=means.dtype))
    return -0.5 * (d * const + logdet + quad)


def normalize_logweights_torch(logw):
    m = torch.max(logw)
    w = torch.exp(logw - m)
    s = torch.sum(w)
    if (not torch.isfinite(s).item()) or (s.item() <= 0):
        return torch.ones_like(logw) / logw.numel()
    return w / s


def systematic_resample_torch(weights, generator=None):
    n = weights.numel()
    u0 = torch.rand((), device=weights.device, dtype=weights.dtype, generator=generator)
    u = (u0 + torch.arange(n, device=weights.device, dtype=weights.dtype)) / n
    cdf = torch.cumsum(weights, dim=0)
    idx = torch.searchsorted(cdf, u, right=True)
    return torch.clamp(idx, 0, n - 1).long()


def lorenz63_rhs_torch(x, sigma=10.0, rho=28.0, beta=8.0 / 3.0):
    xx = x[..., 0]
    yy = x[..., 1]
    zz = x[..., 2]
    return torch.stack([
        sigma * (yy - xx),
        xx * (rho - zz) - yy,
        xx * yy - beta * zz,
    ], dim=-1)


def lorenz63_step_torch(x, dt=0.03):
    k1 = lorenz63_rhs_torch(x)
    k2 = lorenz63_rhs_torch(x + 0.5 * dt * k1)
    k3 = lorenz63_rhs_torch(x + 0.5 * dt * k2)
    k4 = lorenz63_rhs_torch(x + dt * k3)
    return x + (dt / 6.0) * (k1 + 2 * k2 + 2 * k3 + k4)



def symmetrize(M):
    return 0.5 * (M + M.T)


def sqrtm_psd(M, eps=1e-12):
    w, V = np.linalg.eigh(symmetrize(M))
    w = np.clip(w, eps, None)
    return (V * np.sqrt(w)) @ V.T


def invsqrtm_psd(M, eps=1e-12):
    w, V = np.linalg.eigh(symmetrize(M))
    w = np.clip(w, eps, None)
    return (V * (1.0 / np.sqrt(w))) @ V.T


def gaussian_logpdf(x, mean, cov):
    x = np.atleast_1d(x)
    mean = np.atleast_1d(mean)
    cov = np.atleast_2d(cov)
    d = x.shape[0]
    xc = x - mean
    sign, logdet = np.linalg.slogdet(cov)
    if sign <= 0:
        cov = cov + 1e-9 * np.eye(cov.shape[0])
        sign, logdet = np.linalg.slogdet(cov)
    quad = xc @ np.linalg.pinv(cov) @ xc
    return -0.5 * (d * np.log(2.0 * np.pi) + logdet + quad)


def normalize_logweights(logw):
    m = np.max(logw)
    w = np.exp(logw - m)
    s = np.sum(w)
    if (not np.isfinite(s)) or s <= 0:
        return np.ones_like(logw) / len(logw)
    return w / s


def systematic_resample(weights, rng):
    n = len(weights)
    u = (rng.random() + np.arange(n)) / n
    cdf = np.cumsum(weights)
    idx = np.searchsorted(cdf, u, side="right")
    return np.clip(idx, 0, n - 1)


def make_rotation(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s], [s, c]])


def sample_linear_initial(rng, d=2):
    m0 = rng.multivariate_normal(np.zeros(d), np.eye(d))
    v0 = rng.multivariate_normal(m0, np.eye(d))
    return v0, m0


def simulate_linear_gaussian_2d(T=80, theta=0.3, sigma_v=0.01, sigma_y=1.0, seed=0):
    rng = np.random.default_rng(seed)
    A = make_rotation(theta)
    H = np.array([[1.0, 0.0]])
    Sigma = (sigma_v ** 2) * np.eye(2)
    Gamma = np.array([[sigma_y ** 2]])

    truth = np.zeros((T, 2))
    obs = np.zeros((T, 1))

    v, _ = sample_linear_initial(rng, d=2)
    for k in range(T):
        v = A @ v + rng.multivariate_normal(np.zeros(2), Sigma)
        y = H @ v + rng.multivariate_normal(np.zeros(1), Gamma)
        truth[k] = v
        obs[k] = y

    return truth, obs, A, H, Sigma, Gamma


def lorenz63_rhs(x, sigma=10.0, rho=28.0, beta=8.0 / 3.0):
    xx, yy, zz = x
    return np.array([
        sigma * (yy - xx),
        xx * (rho - zz) - yy,
        xx * yy - beta * zz,
    ])


def lorenz63_step(x, dt=0.03):
    k1 = lorenz63_rhs(x)
    k2 = lorenz63_rhs(x + 0.5 * dt * k1)
    k3 = lorenz63_rhs(x + 0.5 * dt * k2)
    k4 = lorenz63_rhs(x + dt * k3)
    return x + (dt / 6.0) * (k1 + 2 * k2 + 2 * k3 + k4)


def lorenz63_burnin_initial(rng, dt=0.03, burn_steps=100):
    m0 = rng.multivariate_normal(np.zeros(3), np.eye(3))
    x = m0.copy()
    for _ in range(burn_steps):
        x = lorenz63_step(x, dt)
    x0 = x.copy()
    v0 = rng.multivariate_normal(x0, np.eye(3))
    return v0, m0, x0


def simulate_lorenz63(T=10.0, dt=0.03, Delta_t=0.15, sigma_v=0.01, sigma_y=1.0, burn_steps=100, seed=0):
    rng = np.random.default_rng(seed)
    n_steps = max(int(T / dt), 1)

    truth = np.zeros((n_steps, 3))
    x, _, _ = lorenz63_burnin_initial(rng, dt=dt, burn_steps=burn_steps)

    for t in range(n_steps):
        x = lorenz63_step(x, dt)
        if sigma_v > 0:
            x = x + rng.normal(0.0, sigma_v, size=3)
        truth[t] = x

    obs_gap = max(int(round(Delta_t / dt)), 1)
    obs_idx = [0]  # Observe first dimension only
    obs_times = np.arange(obs_gap, n_steps, obs_gap)
    Gamma = (sigma_y ** 2) * np.eye(len(obs_idx))
    obs = truth[obs_times][:, obs_idx] + rng.multivariate_normal(np.zeros(len(obs_idx)), Gamma, size=len(obs_times))

    return truth, obs, obs_times, obs_idx


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

plt.ioff()

slider_layout = widgets.Layout(width="280px")
control_row_layout = widgets.Layout(display="flex", flex_flow="row wrap", align_items="center", gap="12px")
button_row_layout = widgets.Layout(display="flex", flex_flow="row", align_items="center", margin="6px 0 0 0")

# -----------------------------
# Linear-Gaussian visualization
# -----------------------------
lg_steps = widgets.IntSlider(value=80, min=30, max=200, step=5, description="steps", layout=slider_layout)
lg_theta = widgets.FloatSlider(value=0.3, min=0.05, max=1.2, step=0.05, description="theta", layout=slider_layout)
lg_sigma_v = widgets.FloatSlider(value=0.01, min=0.001, max=0.2, step=0.001, description="sigma_v", layout=slider_layout)
lg_sigma_y = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description="sigma_y", layout=slider_layout)

lg_plot_btn = widgets.Button(description="Plot Linear", button_style="primary")
lg_out = widgets.Output()

def plot_linear(_):
    """
    Simulates and plots Linear-Gaussian 2D trajectories.
    """
    lg_out.clear_output(wait=True)
    with lg_out:
        truth_lg, obs_lg, A, H, Sigma, Gamma = simulate_linear_gaussian_2d(
            T=lg_steps.value,
            theta=lg_theta.value,
            sigma_v=lg_sigma_v.value,
            sigma_y=lg_sigma_y.value,
        )

        fig, axes = plt.subplots(1, 2, figsize=(14, 4.8))

        axes[0].plot(truth_lg[:, 0], truth_lg[:, 1], color="tab:blue", lw=1.8, label="LG trajectory")
        axes[0].scatter(truth_lg[0, 0], truth_lg[0, 1], color="green", s=50, label="start")
        axes[0].scatter(truth_lg[-1, 0], truth_lg[-1, 1], color="red", s=50, label="end")
        axes[0].set_title("Linear-Gaussian (2D) phase trajectory")
        axes[0].set_xlabel("v1")
        axes[0].set_ylabel("v2")
        axes[0].legend(loc="upper right")

        t_lg = np.arange(lg_steps.value)
        axes[1].plot(t_lg, truth_lg[:, 0], color="black", lw=1.8, label="true v1")
        axes[1].scatter(t_lg, obs_lg[:, 0], color="gray", s=12, alpha=0.55, label="obs y")
        axes[1].set_title("Linear-Gaussian observed channel")
        axes[1].set_xlabel("time index")
        axes[1].set_ylabel("value")
        axes[1].legend(loc="upper right")

        fig.tight_layout()
        plt.show()

# Use explicit click handler management to prevent duplicate callbacks on re-execution
lg_plot_btn.on_click(plot_linear, remove=True)
lg_plot_btn.on_click(plot_linear)

# -----------------------
# Lorenz-63 visualization
# -----------------------
l63_T = widgets.FloatSlider(value=50.0, min=5.0, max=120.0, step=1.0, description="L63 T", layout=slider_layout)
l63_dt = widgets.FloatSlider(value=0.03, min=0.01, max=0.06, step=0.005, description="dt", layout=slider_layout)
l63_Delta_t = widgets.FloatSlider(value=0.15, min=0.03, max=0.30, step=0.03, description="Delta_t", layout=slider_layout)
l63_sigma_v = widgets.FloatSlider(value=0.01, min=0.001, max=0.2, step=0.001, description="sigma_v", layout=slider_layout)
l63_sigma_y = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description="sigma_y", layout=slider_layout)

l63_plot_btn = widgets.Button(description="Plot Lorenz-63", button_style="primary")
l63_out = widgets.Output()

def plot_lorenz63(_):
    """
    Simulates and plots Lorenz-63 chaotic trajectories.
    """
    l63_out.clear_output(wait=True)
    with l63_out:
        truth_l63, obs_l63, obs_times_l63, obs_idx_l63 = simulate_lorenz63(
            T=l63_T.value,
            dt=l63_dt.value,
            Delta_t=l63_Delta_t.value,
            sigma_v=l63_sigma_v.value,
            sigma_y=l63_sigma_y.value,
            burn_steps=100,
            seed=0,
        )

        t_l63 = np.arange(len(truth_l63)) * l63_dt.value
        fig = plt.figure(figsize=(14, 4.8))

        ax1 = fig.add_subplot(1, 2, 1, projection="3d")
        ax1.plot(truth_l63[:, 0], truth_l63[:, 1], truth_l63[:, 2], color="tab:purple", lw=1.3)
        ax1.set_title("Lorenz-63 trajectory")
        ax1.set_xlabel("x")
        ax1.set_ylabel("y")
        ax1.set_zlabel("z")

        ax2 = fig.add_subplot(1, 2, 2)
        ax2.plot(t_l63, truth_l63[:, 0], color="tab:blue", lw=1.6, label="true x")
        ax2.scatter(obs_times_l63 * l63_dt.value, obs_l63[:, 0], color="tab:blue", s=14, alpha=0.5, label="obs x")
        ax2.set_title("Lorenz-63 observed channel (x only)")
        ax2.set_xlabel("time")
        ax2.set_ylabel("value")
        ax2.legend(loc="upper right")

        fig.tight_layout()
        plt.show()

# Reset and bind callback to ensure singular event execution
l63_plot_btn.on_click(plot_lorenz63, remove=True)
l63_plot_btn.on_click(plot_lorenz63)

# -----------------
# Final UI Display
# -----------------
display(widgets.HTML("<h4>Linear Controls</h4>"))
display(widgets.VBox([
    widgets.HBox([lg_steps, lg_theta, lg_sigma_v, lg_sigma_y], layout=control_row_layout),
    widgets.HBox([lg_plot_btn], layout=button_row_layout),
]))
display(lg_out)

display(widgets.HTML("<h4>Lorenz-63 Controls</h4>"))
display(widgets.VBox([
    widgets.HBox([l63_T, l63_dt, l63_Delta_t, l63_sigma_v, l63_sigma_y], layout=control_row_layout),
    widgets.HBox([l63_plot_btn], layout=button_row_layout),
]))
display(l63_out)

## 2. Particle Filters

At time step $j$, we assume the filtering measure $\pi_j$ is approximated by a set of $N$ **equally weighted** particles $\{v_j^{(n)}\}_{n=1}^N$:
$$
\pi_j \approx \pi_j^N = \frac{1}{N} \sum_{n=1}^N \delta_{v_j^{(n)}}.
$$
Our goal is to approximate the next filtering distribution $\pi_{j+1} = \mathbb{P}(V_{j+1}^{\dagger} \mid Y_{j+1}^{\dagger})$ by a new set of particles $\{v_{j+1}^{(n)}\}_{n=1}^N$. Since we cannot sample from the target $\pi_{j+1}$ directly, we use Importance Sampling. This involves introducing a proposal distribution $q(v_{j+1} \mid v_j^{(n)}, y_{j+1})$, an easy-to-sample density that guides the particles toward regions of high posterior probability.

### 2.1 Bootstrap Particle Filter (BPF)
The BPF is the most straightforward implementation of the sequential importance sampling and resampling (SIR) framework. It uses the transition prior as the proposal distribution.

1.  **Propagation:** For each particle $n$, sample a candidate from the transition density:
$$
\hat{v}_{j+1}^{(n)} \sim p(v_{j+1} \mid v_j^{(n)}) = \mathcal{N}(\Psi(v_j^{(n)}), \Sigma).
$$
2.  **Weighting:** Assign importance weights based on the likelihood of the new observation $y_{j+1}$:
$$
w_{j+1}^{(n)} \propto p(y_{j+1} \mid \hat{v}_{j+1}^{(n)}) = \mathcal{N}(y_{j+1}; h(\hat{v}_{j+1}^{(n)}), \Gamma).
$$
3.  **Resampling:** Sample $N$ new particles $\{v_{j+1}^{(n)}\}_{n=1}^N$ from the set $\{\hat{v}_{j+1}^{(k)}\}_{k=1}^N$ with probabilities proportional to $w_{j+1}^{(k)}$. After resampling, the particles are again equally weighted with $1/N$.



### 2.2 Optimal Particle Filter (OPF)
The weights in BPF often become unevenly distributed (high variance) because the proposal does not account for the current observation $y_{j+1}$. The OPF addresses this by choosing a proposal that minimizes the variance of the incremental weights:
$$
q^\star(v_{j+1} \mid v_j^{(n)}, y_{j+1}) = p(v_{j+1} \mid v_j^{(n)}, y_{j+1}).
$$

#### 2.2.1 Linear-Gaussian Case
For the linear model (Test Dataset 1), the optimal proposal is Gaussian and can be computed in closed form:
$$
q^\star(v_{j+1} \mid v_j^{(n)}, y_{j+1}) = \mathcal{N}(m_{j+1}^{(n)}, C^\star),
$$
$$
C^\star = (\Sigma^{-1} + H^T \Gamma^{-1} H)^{-1}, \quad m_{j+1}^{(n)} = C^\star (\Sigma^{-1} \Psi(v_j^{(n)}) + H^T \Gamma^{-1} y_{j+1}).
$$
The incremental weights are proportional to the predictive likelihood:
$$
w_{j+1}^{(n)} \propto p(y_{j+1} \mid v_j^{(n)}) = \mathcal{N}(y_{j+1}; h(\Psi(v_j^{(n)})), H \Sigma H^T + \Gamma).
$$

#### 2.2.2 General Case (Nonlinear Approximation)
For nonlinear models like Lorenz-63, the optimal proposal is usually intractable. A common approach is to approximate $p(v_{j+1} \mid v_j^{(n)}, y_{j+1})$ with a local Gaussian centered at the MAP (3DVar) point:
$$
\hat{v}_{j+1}^{(n)} = \arg \min_v \left( \frac{1}{2} \| v - \Psi(v_j^{(n)}) \|_{\Sigma}^2 + \frac{1}{2} \| y_{j+1} - h(v) \|_{\Gamma}^2 \right).
$$
The proposal is then $\mathcal{N}(\hat{v}_{j+1}^{(n)}, \hat{C}_n)$, where $\hat{C}_n$ is the inverse Hessian of the objective function.

### 2.3 Auxiliary Particle Filter (APF)
The APF introduces a "look-ahead" mechanism to identify promising ancestors before the propagation step. It effectively changes the order of operations by resampling based on a proxy of the future likelihood.

#### 2.3.1 Stage 1: Ancestor Selection
We first evaluate the potential of each current particle $v_j^{(n)}$ by using a point estimate (proxy) of its future state, such as the mean transition $\bar{v}_{j+1}^{(n)} = \Psi(v_j^{(n)})$. We calculate ancestor scores:
$$
\tilde{w}_j^{(n)} \propto \frac{1}{N} p(y_{j+1} \mid \bar{v}_{j+1}^{(n)}) = \frac{1}{N} \mathcal{N}(y_{j+1}; h(\Psi(v_j^{(n)})), \Gamma).
$$
We then resample $N$ ancestor indices $\{a^{(n)}\}_{n=1}^N$ from $\{1, \dots, N\}$ according to these scores $\tilde{w}_j$. This ensures we only propagate particles that are likely to be consistent with $y_{j+1}$.

#### 2.3.2 Stage 2: Propagation and Correction
For each selected ancestor index $a^{(n)}$, we sample the new state from the transition prior:
$$
v_{j+1}^{(n)} \sim p(v_{j+1} \mid v_j^{(a^{(n)})}) = \mathcal{N}(\Psi(v_j^{(a^{(n)})}), \Sigma).
$$
Because the ancestors were selected using the proxy $\bar{v}_{j+1}$, we must apply a correction weight to maintain a consistent approximation of the filtering measure $\pi_{j+1}$:
$$
w_{j+1}^{(n)} \propto \frac{p(y_{j+1} \mid v_{j+1}^{(n)})}{p(y_{j+1} \mid \bar{v}_{j+1}^{(a^{(n)})})}.
$$
This weight adjusts for the discrepancy between the actual sample $v_{j+1}^{(n)}$ and the look-ahead proxy $\bar{v}_{j+1}$ used in Stage 1.

In [ ]:
# -------------------------------------------------------------------------
# Particle Filter Runners (using your provided helper functions)
# -------------------------------------------------------------------------

def run_pf_linear_torch(obs, A, H, Sigma, Gamma, N, method, use_true_init, truth_x0, seed, progress_bar=None):
    device = torch.device("cuda")
    torch.manual_seed(int(seed))
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(int(seed))

    obs_t = as_torch(obs, device=device)
    A_t = as_torch(A, device=device)
    H_t = as_torch(H, device=device)
    Sigma_t = as_torch(Sigma, device=device)
    Gamma_t = as_torch(Gamma, device=device)
    truth_x0_t = as_torch(truth_x0, device=device)

    T = obs_t.shape[0]
    d = A_t.shape[0]

    if use_true_init:
        particles = mvn_sample_torch(truth_x0_t, torch.eye(d, device=device, dtype=obs_t.dtype), size=N)
    else:
        epsilon = torch.randn(d, device=device, dtype=obs_t.dtype)
        perturbed_x0 = truth_x0_t * (1.0 + 0.3 * epsilon)
        particles = mvn_sample_torch(perturbed_x0, torch.eye(d, device=device, dtype=obs_t.dtype), size=N)

    weights = torch.full((N,), 1.0 / N, device=device, dtype=obs_t.dtype)
    est = torch.zeros((T, d), device=device, dtype=obs_t.dtype)

    Sigma_inv = torch.linalg.pinv(Sigma_t)
    Gamma_inv = torch.linalg.pinv(Gamma_t)
    C_opt = torch.linalg.pinv(Sigma_inv + H_t.T @ Gamma_inv @ H_t)
    S_pred = H_t @ Sigma_t @ H_t.T + Gamma_t

    zeros_d = torch.zeros(d, device=device, dtype=obs_t.dtype)

    for k in range(T):
        y = obs_t[k].reshape(-1)

        if method == "bootstrap":
            pred_mean = particles @ A_t.T
            prop = pred_mean + mvn_sample_torch(zeros_d, Sigma_t, size=N)
            logw = torch.log(weights + 1e-300)
            logw = logw + gaussian_logpdf_batch_torch(y, prop @ H_t.T, Gamma_t)
            particles = prop

        elif method == "optimal":
            prev = particles.clone()
            pred_mean = prev @ A_t.T
            prop = torch.zeros_like(prev)
            logw = torch.log(weights + 1e-300)
            obs_term = H_t.T @ Gamma_inv @ y

            for i in range(N):
                m_star = C_opt @ (Sigma_inv @ pred_mean[i] + obs_term)
                prop[i] = mvn_sample_torch(m_star, C_opt)

            logw = logw + gaussian_logpdf_batch_torch(y, pred_mean @ H_t.T, S_pred)
            particles = prop

        elif method == "auxiliary":
            prev = particles.clone()
            pred_mean = prev @ A_t.T

            first_logw = torch.log(weights + 1e-300)
            first_logw = first_logw + gaussian_logpdf_batch_torch(y, pred_mean @ H_t.T, S_pred)
            first_w = normalize_logweights_torch(first_logw)
            ancestors = torch.multinomial(first_w, N, replacement=True)

            anc_mean = pred_mean[ancestors]
            prop = anc_mean + mvn_sample_torch(zeros_d, Sigma_t, size=N)

            logw = gaussian_logpdf_batch_torch(y, prop @ H_t.T, Gamma_t)
            logw = logw - gaussian_logpdf_batch_torch(y, anc_mean @ H_t.T, S_pred)
            particles = prop
        else:
            raise ValueError(f"Unknown method: {method}")

        weights = normalize_logweights_torch(logw)
        est[k] = torch.sum(weights[:, None] * particles, dim=0)

        ess = 1.0 / torch.sum(weights ** 2)
        if ess.item() < 0.5 * N:
            idx = systematic_resample_torch(weights)
            particles = particles[idx]
            weights.fill_(1.0 / N)

        if progress_bar:
            progress_bar.value += 1

    return est.detach().cpu().numpy()


def run_pf_linear(obs, A, H, Sigma, Gamma, N, method, use_true_init, truth_x0, seed, progress_bar=None):
    """
    Particle Filter for Linear-Gaussian system using provided helper functions.
    input: obs (T, d_obs), system matrices, N (particles), method (str), init flags, truth_x0 (d,), seed
    output: est (T, d) state estimates
    """
    if TORCH_GPU_AVAILABLE:
        return run_pf_linear_torch(obs, A, H, Sigma, Gamma, N, method, use_true_init, truth_x0, seed, progress_bar=progress_bar)

    rng = np.random.default_rng(seed)
    T = len(obs)
    d = A.shape[0]

    # Initialization
    if use_true_init:
        # N(v0, I)
        particles = rng.multivariate_normal(truth_x0, np.eye(d), size=N)
    else:
        # v0_tilde = v0 * (1 + 0.3 * epsilon), epsilon ~ N(0, I)
        epsilon = rng.standard_normal(size=d)
        perturbed_x0 = truth_x0 * (1 + 0.3 * epsilon)
        particles = rng.multivariate_normal(perturbed_x0, np.eye(d), size=N)

    weights = np.ones(N) / N
    est = np.zeros((T, d))

    # Pre-compute matrices for Optimal PF (Linear Case)
    Sigma_inv = np.linalg.pinv(Sigma)
    Gamma_inv = np.linalg.pinv(Gamma)
    # C_opt: Posterior covariance for the optimal proposal
    C_opt = np.linalg.pinv(Sigma_inv + H.T @ Gamma_inv @ H)
    # S_pred: Predictive covariance p(y_t | x_{t-1})
    S_pred = H @ Sigma @ H.T + Gamma

    for k in range(T):
        y = obs[k]

        # --- Bootstrap PF ---
        if method == "bootstrap":
            # Prediction: x_t = A x_{t-1}
            pred_mean = particles @ A.T
            # Propagate noise
            prop = pred_mean + rng.multivariate_normal(np.zeros(d), Sigma, size=N)

            # Weight update
            logw = np.log(weights + 1e-300)
            for i in range(N):
                logw[i] += gaussian_logpdf(y, H @ prop[i], Gamma)
            particles = prop

        # --- Optimal PF (Exact) ---
        elif method == "optimal":
            prev = particles.copy()
            pred_mean = prev @ A.T
            prop = np.zeros_like(prev)
            logw = np.log(weights + 1e-300)

            for i in range(N):
                # Optimal proposal mean: m* = C_opt (Sigma^-1 A x_{t-1} + H^T Gamma^-1 y_t)
                m_star = C_opt @ (Sigma_inv @ pred_mean[i] + H.T @ Gamma_inv @ y)
                prop[i] = rng.multivariate_normal(m_star, C_opt)

                # Weight update uses predictive density directly
                logw[i] += gaussian_logpdf(y, H @ pred_mean[i], S_pred)
            particles = prop

        # --- Auxiliary PF ---
        elif method == "auxiliary":
            prev = particles.copy()
            pred_mean = prev @ A.T

            # 1. Lookahead / Pilot weights
            first_logw = np.log(weights + 1e-300)
            for i in range(N):
                first_logw[i] += gaussian_logpdf(y, H @ pred_mean[i], S_pred)

            # 2. Resample indices (Auxiliary variable)
            first_w = normalize_logweights(first_logw)
            ancestors = rng.choice(N, size=N, p=first_w)

            anc_mean = pred_mean[ancestors]

            # 3. Propagate
            prop = anc_mean + rng.multivariate_normal(np.zeros(d), Sigma, size=N)

            # 4. Correct weights
            logw = np.zeros(N)
            for i in range(N):
                likelihood = gaussian_logpdf(y, H @ prop[i], Gamma)
                prior = gaussian_logpdf(y, H @ anc_mean[i], S_pred)
                logw[i] = likelihood - prior
            particles = prop

        # Estimate and ESS
        weights = normalize_logweights(logw)
        est[k] = np.sum(weights[:, None] * particles, axis=0)

        # Resampling check
        ess = 1.0 / np.sum(weights ** 2)
        if ess < 0.5 * N:
            idx = systematic_resample(weights, rng)
            particles = particles[idx]
            weights.fill(1.0 / N)

        if progress_bar:
            progress_bar.value += 1

    return est


def run_pf_lorenz_torch(obs, obs_times, obs_idx, dt, sigma_v, sigma_y, N, method, use_true_init, truth_x0, seed, progress_bar=None):
    device = torch.device("cuda")
    torch.manual_seed(int(seed))
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(int(seed))

    T_obs = len(obs)
    d = 3

    H_t = torch.zeros((len(obs_idx), d), device=device, dtype=TORCH_DTYPE)
    for r, c in enumerate(obs_idx):
        H_t[r, c] = 1.0

    Gamma_t = (sigma_y ** 2) * torch.eye(len(obs_idx), device=device, dtype=TORCH_DTYPE)
    Gamma_inv = torch.linalg.pinv(Gamma_t)

    obs_gap = int(obs_times[0]) if len(obs_times) > 0 else 1
    Q_val = max(obs_gap * (sigma_v ** 2), 1e-6)
    Q_t = Q_val * torch.eye(d, device=device, dtype=TORCH_DTYPE)
    Q_inv = torch.linalg.pinv(Q_t)
    C_opt = torch.linalg.pinv(Q_inv + H_t.T @ Gamma_inv @ H_t)
    S_pred = H_t @ Q_t @ H_t.T + Gamma_t

    truth_x0_t = as_torch(truth_x0, device=device, dtype=TORCH_DTYPE)
    if use_true_init:
        particles = mvn_sample_torch(truth_x0_t, torch.eye(d, device=device, dtype=TORCH_DTYPE), size=N)
    else:
        epsilon = torch.randn(d, device=device, dtype=TORCH_DTYPE)
        perturbed_x0 = truth_x0_t * (1.0 + 0.3 * epsilon)
        particles = mvn_sample_torch(perturbed_x0, torch.eye(d, device=device, dtype=TORCH_DTYPE), size=N)

    obs_t = as_torch(obs, device=device, dtype=TORCH_DTYPE)
    weights = torch.full((N,), 1.0 / N, device=device, dtype=TORCH_DTYPE)
    est = torch.zeros((T_obs, d), device=device, dtype=TORCH_DTYPE)
    current_time_step = 0

    for k in range(T_obs):
        target_time_step = int(obs_times[k])
        steps_to_sim = max(target_time_step - current_time_step, 0)
        y = obs_t[k].reshape(-1)

        prev = particles.clone()
        pred_mean = prev.clone()
        for _ in range(steps_to_sim):
            pred_mean = lorenz63_step_torch(pred_mean, dt)

        if method == "bootstrap":
            prop = prev.clone()
            for _ in range(steps_to_sim):
                prop = lorenz63_step_torch(prop, dt)
                if sigma_v > 0:
                    prop = prop + sigma_v * torch.randn_like(prop)

            logw = torch.log(weights + 1e-300)
            logw = logw + gaussian_logpdf_batch_torch(y, prop @ H_t.T, Gamma_t)
            particles = prop

        elif method == "optimal_3dvar":
            prop = torch.zeros_like(prev)
            logw = torch.log(weights + 1e-300)
            obs_term = H_t.T @ Gamma_inv @ y

            for i in range(N):
                m_star = C_opt @ (Q_inv @ pred_mean[i] + obs_term)
                prop[i] = mvn_sample_torch(m_star, C_opt)

            logw = logw + gaussian_logpdf_batch_torch(y, pred_mean @ H_t.T, S_pred)
            particles = prop

        elif method == "auxiliary":
            first_logw = torch.log(weights + 1e-300)
            first_logw = first_logw + gaussian_logpdf_batch_torch(y, pred_mean @ H_t.T, S_pred)

            first_w = normalize_logweights_torch(first_logw)
            ancestors = torch.multinomial(first_w, N, replacement=True)

            anc_mean = pred_mean[ancestors]
            anc_particles = prev[ancestors]

            prop = anc_particles.clone()
            for _ in range(steps_to_sim):
                prop = lorenz63_step_torch(prop, dt)
                if sigma_v > 0:
                    prop = prop + sigma_v * torch.randn_like(prop)

            logw = gaussian_logpdf_batch_torch(y, prop @ H_t.T, Gamma_t)
            logw = logw - gaussian_logpdf_batch_torch(y, anc_mean @ H_t.T, S_pred)
            particles = prop
        else:
            raise ValueError(f"Unknown method: {method}")

        weights = normalize_logweights_torch(logw)
        est[k] = torch.sum(weights[:, None] * particles, dim=0)

        if (1.0 / torch.sum(weights ** 2)).item() < 0.5 * N:
            idx = systematic_resample_torch(weights)
            particles = particles[idx]
            weights.fill_(1.0 / N)

        current_time_step = target_time_step

        if progress_bar:
            progress_bar.value += 1

    return est.detach().cpu().numpy()


def run_pf_lorenz(obs, obs_times, obs_idx, dt, sigma_v, sigma_y, N, method, use_true_init, truth_x0, seed, progress_bar=None):
    """
    Particle Filter for Lorenz-63 using the provided RK4 'lorenz63_step'.
    input: obs, times, indices, system params, N, method, init flags, truth_x0, seed
    output: est (T_obs, 3) state estimates at observation times
    """
    if TORCH_GPU_AVAILABLE:
        return run_pf_lorenz_torch(
            obs,
            obs_times,
            obs_idx,
            dt,
            sigma_v,
            sigma_y,
            N,
            method,
            use_true_init,
            truth_x0,
            seed,
            progress_bar=progress_bar,
        )

    rng = np.random.default_rng(seed)
    T_obs = len(obs)
    d = 3 # State dimension

    # Construct H based on obs_idx (e.g., [0] -> [[1, 0, 0]])
    H = np.zeros((len(obs_idx), d))
    for r, c in enumerate(obs_idx):
        H[r, c] = 1.0

    Gamma = (sigma_y ** 2) * np.eye(len(obs_idx))
    Gamma_inv = np.linalg.pinv(Gamma)

    # Calculate obs_gap from times (assuming uniform gap)
    obs_gap = obs_times[0] if len(obs_times) > 0 else 1

    # Approximate Q for proposal design (Linearized approx)
    Q_val = max(obs_gap * (sigma_v**2), 1e-6)
    Q = Q_val * np.eye(d)
    Q_inv = np.linalg.pinv(Q)

    # Pre-calc approximate optimal matrices
    C_opt = np.linalg.pinv(Q_inv + H.T @ Gamma_inv @ H)
    S_pred = H @ Q @ H.T + Gamma

    # Initialization
    if use_true_init:
        particles = rng.multivariate_normal(truth_x0, np.eye(d), size=N)
    else:
        # v0_tilde = v0 * (1 + 0.3 * epsilon), epsilon ~ N(0, I)
        epsilon = rng.standard_normal(size=d)
        perturbed_x0 = truth_x0 * (1 + 0.3 * epsilon)
        particles = rng.multivariate_normal(perturbed_x0, np.eye(d), size=N)

    weights = np.ones(N) / N
    est = np.zeros((T_obs, d))

    current_time_step = 0

    for k in range(T_obs):
        target_time_step = obs_times[k]
        steps_to_sim = target_time_step - current_time_step
        y = np.atleast_1d(obs[k])

        prev = particles.copy()
        pred_mean = np.zeros_like(prev)

        # 1. Deterministic Forecast (for mean-based proposals)
        # We use the provided lorenz63_step (RK4)
        for i in range(N):
            temp = prev[i]
            for _ in range(steps_to_sim):
                temp = lorenz63_step(temp, dt)
            pred_mean[i] = temp

        # --- Bootstrap ---
        if method == "bootstrap":
            prop = np.zeros_like(prev)
            for i in range(N):
                temp = prev[i]
                for _ in range(steps_to_sim):
                    temp = lorenz63_step(temp, dt)
                    if sigma_v > 0:
                        temp += rng.normal(0, sigma_v, size=d)
                prop[i] = temp

            logw = np.log(weights + 1e-300)
            for i in range(N):
                logw[i] += gaussian_logpdf(y, H @ prop[i], Gamma)
            particles = prop

        # --- Optimal (Approx 3DVar) ---
        elif method == "optimal_3dvar":
            prop = np.zeros_like(prev)
            logw = np.log(weights + 1e-300)

            for i in range(N):
                # Use the deterministic forecast 'pred_mean' as the prior mean for this step
                m_star = C_opt @ (Q_inv @ pred_mean[i] + H.T @ Gamma_inv @ y)
                prop[i] = rng.multivariate_normal(m_star, C_opt)
                logw[i] += gaussian_logpdf(y, H @ pred_mean[i], S_pred)
            particles = prop

        # --- Auxiliary ---
        elif method == "auxiliary":
            # Stage 1: Weight based on predictive mean
            first_logw = np.log(weights + 1e-300)
            for i in range(N):
                first_logw[i] += gaussian_logpdf(y, H @ pred_mean[i], S_pred)

            first_w = normalize_logweights(first_logw)
            ancestors = rng.choice(N, size=N, p=first_w)

            anc_mean = pred_mean[ancestors]
            anc_particles = prev[ancestors]

            # Stage 2: Propagate ancestors
            prop = np.zeros_like(prev)
            for i in range(N):
                temp = anc_particles[i]
                for _ in range(steps_to_sim):
                    temp = lorenz63_step(temp, dt)
                    if sigma_v > 0:
                        temp += rng.normal(0, sigma_v, size=d)
                prop[i] = temp

            # Stage 3: Reweight
            logw = np.zeros(N)
            for i in range(N):
                lik = gaussian_logpdf(y, H @ prop[i], Gamma)
                prior = gaussian_logpdf(y, H @ anc_mean[i], S_pred)
                logw[i] = lik - prior
            particles = prop

        # Estimate
        weights = normalize_logweights(logw)
        est[k] = np.sum(weights[:, None] * particles, axis=0)

        # Resample
        if 1.0 / np.sum(weights**2) < 0.5 * N:
            idx = systematic_resample(weights, rng)
            particles = particles[idx]
            weights.fill(1.0 / N)

        # Update state
        current_time_step = target_time_step

        if progress_bar:
            progress_bar.value += 1

    return est


# -------------------------------------------------------------------------
# UI & Plotting Logic
# -------------------------------------------------------------------------

# Widget Layouts
style_layout = widgets.Layout(width="280px")
btn_layout = widgets.Layout(width="120px")
progress_layout = widgets.Layout(width="300px", margin="5px 0 0 10px")

# --- Linear Widgets ---
lg_particles = widgets.IntSlider(value=100, min=50, max=1000, step=50, description="Particles (N)", layout=style_layout)
lg_init = widgets.Checkbox(value=True, description="Use True Init N(v0, I)")
lg_btn = widgets.Button(description="Run Linear", button_style='info', icon='play', layout=btn_layout)
lg_progress = widgets.IntProgress(value=0, min=0, max=100, description='Progress:', bar_style='info', layout=progress_layout)
lg_out = widgets.Output()

# --- Lorenz Widgets ---
l63_particles = widgets.IntSlider(value=100, min=50, max=1000, step=50, description="Particles (N)", layout=style_layout)
l63_init = widgets.Checkbox(value=True, description="Use True Init N(v0, I)")
l63_btn = widgets.Button(description="Run Lorenz", button_style='info', icon='play', layout=btn_layout)
l63_progress = widgets.IntProgress(value=0, min=0, max=100, description='Progress:', bar_style='info', layout=progress_layout)
l63_out = widgets.Output()


def plot_linear_callback(_):
    lg_out.clear_output(wait=True)

    # Simulation Params
    T_sim = 80
    theta = 0.3
    sv = 0.01
    sy = 1.0
    SEED = 42

    # PF Params
    N = lg_particles.value
    use_init = lg_init.value

    methods = [
        ("Bootstrap", "bootstrap", "tab:blue"),
        ("Optimal", "optimal", "tab:green"),
        ("Auxiliary", "auxiliary", "tab:orange")
    ]

    # Setup Progress Bar
    lg_progress.value = 0
    lg_progress.max = len(methods) * T_sim
    lg_progress.description = "Running..."

    with lg_out:
        # 1. Generate Truth using provided function
        truth, obs, A, H, Sigma, Gamma = simulate_linear_gaussian_2d(T=T_sim, theta=theta, sigma_v=sv, sigma_y=sy, seed=SEED)

        results = []
        rmses = []

        # 2. Run PFs
        for name, m_code, color in methods:
            # We use truth[0] as the initialization center
            est = run_pf_linear(obs, A, H, Sigma, Gamma, N, m_code, use_init, truth[0], seed=SEED+10, progress_bar=lg_progress)

            # RMSE Calculation
            rmse = np.sqrt(np.mean(np.sum((est - truth)**2, axis=1)))
            results.append((name, est, color))
            rmses.append(rmse)

        lg_progress.description = "Done"

        # 3. Plotting
        fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

        # Phase Plot
        ax = axes[0]
        ax.plot(truth[:,0], truth[:,1], 'k-', lw=2, label='Truth')
        for name, est, col in results:
            ax.plot(est[:,0], est[:,1], color=col, lw=1, alpha=0.8, label=name)
        ax.set_title("2D Phase Trajectory")
        ax.legend()

        # Obs Dim Plot
        ax = axes[1]
        t = np.arange(T_sim)
        ax.plot(t, truth[:,0], 'k-', label='Truth (Obs Dim)')
        ax.scatter(t, obs, c='gray', s=10, alpha=0.5, label='Obs')
        for name, est, col in results:
            ax.plot(t, est[:,0], color=col, lw=1, alpha=0.8)
        ax.set_title("Observed Dimension vs Time")

        # RMSE Plot
        ax = axes[2]
        bars = ax.bar([r[0] for r in methods], rmses, color=[r[2] for r in methods])
        ax.set_title("RMSE (Estimate vs Truth)")
        ax.set_ylabel("RMSE")
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.3f}', ha='center', va='bottom')

        plt.tight_layout()
        plt.show()


def plot_lorenz_callback(_):
    l63_out.clear_output(wait=True)

    # Simulation Params
    T_sim = 30.0 # Time units
    dt = 0.03
    Delta_t = 0.15 # Observation gap
    sv = 0.01
    sy = 1.0
    SEED = 99

    # PF Params
    N = l63_particles.value
    use_init = l63_init.value

    methods = [
        ("Bootstrap", "bootstrap", "tab:blue"),
        ("Approx Opt (3DVar)", "optimal_3dvar", "tab:green"),
        ("Auxiliary", "auxiliary", "tab:orange")
    ]

    with l63_out:
        # 1. Generate Truth using provided function
        # Returns: truth (steps, 3), obs (n_obs, 1), obs_times (n_obs,), obs_idx list
        truth_full, obs, obs_times, obs_idx = simulate_lorenz63(
            T=T_sim, dt=dt, Delta_t=Delta_t, sigma_v=sv, sigma_y=sy, burn_steps=100, seed=SEED
        )

        truth_at_obs = truth_full[obs_times]

        # Setup Progress Bar
        l63_progress.value = 0
        l63_progress.max = len(methods) * len(obs)
        l63_progress.description = "Running..."

        results = []
        rmses = []

        # 2. Run PFs
        for name, m_code, color in methods:
            # Init at truth_full[0] (state at t=0)
            est = run_pf_lorenz(
                obs, obs_times, obs_idx, dt, sv, sy, N,
                m_code, use_init, truth_full[0], seed=SEED+50, progress_bar=l63_progress
            )

            # RMSE against truth at observation times
            rmse = np.sqrt(np.mean(np.sum((est - truth_at_obs)**2, axis=1)))
            results.append((name, est, color))
            rmses.append(rmse)

        l63_progress.description = "Done"

        # 3. Plotting
        fig = plt.figure(figsize=(16, 5))

        # 3D Trajectory
        ax = fig.add_subplot(1, 3, 1, projection='3d')
        ax.plot(truth_full[:,0], truth_full[:,1], truth_full[:,2], 'k-', lw=0.5, alpha=0.3, label='Full Truth')
        for name, est, col in results:
            ax.plot(est[:,0], est[:,1], est[:,2], color=col, lw=1.5, label=name)
        ax.set_title("3D Trajectory (Estimates)")
        ax.legend()

        # Observed Dim X
        ax = fig.add_subplot(1, 3, 2)
        t_full = np.arange(len(truth_full)) * dt
        t_obs = obs_times * dt
        ax.plot(t_full, truth_full[:,0], 'k-', lw=1, alpha=0.5, label='Truth X')
        for name, est, col in results:
            ax.plot(t_obs, est[:,0], color=col, marker='o', markersize=3, lw=1, label=name)
        ax.set_title("Observed Dimension X vs Time")
        ax.legend()

        # RMSE Bar
        ax = fig.add_subplot(1, 3, 3)
        names = [r[0] for r in methods]
        bars = ax.bar(names, rmses, color=[r[2] for r in methods])
        ax.set_title("RMSE (Estimate vs Truth)")
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.3f}', ha='center', va='bottom')

        plt.tight_layout()
        plt.show()


# Bind callbacks
lg_btn.on_click(plot_linear_callback)
l63_btn.on_click(plot_lorenz_callback)

# --- Final Display ---
display(widgets.HTML("<h3>System 1: Linear Gaussian (2D)</h3>"))
display(widgets.HBox([lg_particles, lg_init, lg_btn, lg_progress]))
display(lg_out)

display(widgets.HTML("<hr><h3>System 2: Lorenz-63 (Chaotic)</h3>"))
display(widgets.HBox([l63_particles, l63_init, l63_btn, l63_progress]))
display(l63_out)



## 3. Gaussian Filters from an Affine Analysis Map

In this section, we first derive the linear Kalman update using the cleanest covariance notation, then embed it into an affine analysis map.

### 3.1 Kalman update from predictive moments

At time $j+1$, let the predictive (prior) state be
$$
\widehat v_{j+1} \sim \mathcal N(\widehat m_{j+1},\widehat C_{j+1}),
$$
and define predictive observation
$$
\widehat y_{j+1} = H\widehat v_{j+1} + \eta_{j+1}, \qquad \eta_{j+1}\sim\mathcal N(0,\Gamma).
$$
So
$$
\widehat m_{j+1}^{y}=\mathbb E[\widehat y_{j+1}] = H\widehat m_{j+1},
\qquad
\widehat C_{j+1}^{vy}=\mathrm{Cov}(\widehat v_{j+1},\widehat y_{j+1}),
\qquad
\widehat C_{j+1}^{yy}=\mathrm{Cov}(\widehat y_{j+1},\widehat y_{j+1}).
$$
In the linear-Gaussian case,
$$
\widehat C_{j+1}^{vy}=\widehat C_{j+1}H^T,
\qquad
\widehat C_{j+1}^{yy}=H\widehat C_{j+1}H^T+\Gamma.
$$

Assume analysis uses the increment form
$$
v_{j+1} = \widehat v_{j+1} + K_{j+1}\big(y_{j+1}^{\dagger} - h(\widehat v_{j+1})\big).
$$
For $h(v)=Hv$,
$$
v_{j+1} = \widehat v_{j+1} + K_{j+1}\big(y_{j+1}^{\dagger} - H\widehat v_{j+1}\big).
$$
Matching Gaussian posterior moments gives
$$
m_{j+1} = \widehat m_{j+1} + K_{j+1}\big(y_{j+1}^{\dagger} - \widehat m_{j+1}^{y}\big),
$$
$$
C_{j+1} = (I-K_{j+1}H)\widehat C_{j+1}(I-K_{j+1}H)^T + K_{j+1}\Gamma K_{j+1}^T.
$$
and the Kalman gain is written directly as
$$
K_{j+1}=\widehat C_{j+1}^{vy}\,(\widehat C_{j+1}^{yy})^{-1}.
$$
In matrix form for this testbed,
$$
K_{j+1}=\widehat C_{j+1}H^T\big(H\widehat C_{j+1}H^T+\Gamma\big)^{-1}.
$$

### 3.2 Affine analysis map viewpoint

Now write a general affine analysis map
$$
v_{j+1}=\mathcal A_{j+1}(\widehat\pi_{j+1},y_{j+1}^{\dagger})\widehat v_{j+1}
+\mathcal B_{j+1}(\widehat\pi_{j+1},y_{j+1}^{\dagger})\widehat y_{j+1}
+a_{j+1}(\widehat\pi_{j+1},y_{j+1}^{\dagger}).
$$
Imposing exactness with the Kalman target moments gives
$$
a_{j+1}=(I-\mathcal A_{j+1})\,\widehat m_{j+1}
+K_{j+1}y_{j+1}^{\dagger}
-(\mathcal B_{j+1}+K_{j+1})\,\widehat m_{j+1}^{y}.
$$
For Kalman filter,
$$
\mathcal A_{j+1}=I,\ \mathcal B_{j+1}=-K_{j+1}\quad\Rightarrow\quad\text{Kalman filter},
$$
and other Gaussian filters can be interpreted as alternative choices of $(\mathcal A_{j+1},\mathcal B_{j+1})$ that preserve the same analysis moments.
In this section's code, we only implement and visualize the linear Kalman filter.


In [ ]:
from IPython.display import display


def run_affine_gaussian_filter_torch(obs, A, H, Sigma, Gamma, m0=None, C0=None, return_gain=False):
    device = torch.device("cuda")

    obs_t = as_torch(obs, device=device)
    A_t = as_torch(A, device=device)
    H_t = as_torch(H, device=device)
    Sigma_t = as_torch(Sigma, device=device)
    Gamma_t = as_torch(Gamma, device=device)

    T = obs_t.shape[0]
    d = A_t.shape[0]
    m_dim = H_t.shape[0]

    if m0 is None:
        m0 = np.zeros(d)
    if C0 is None:
        C0 = 2.0 * np.eye(d)

    m = as_torch(m0, device=device, dtype=obs_t.dtype)
    C = as_torch(C0, device=device, dtype=obs_t.dtype)

    m_hist = torch.zeros((T, d), device=device, dtype=obs_t.dtype)
    C_hist = torch.zeros((T, d, d), device=device, dtype=obs_t.dtype)
    K_hist = torch.zeros((T, d, m_dim), device=device, dtype=obs_t.dtype)

    I = torch.eye(d, device=device, dtype=obs_t.dtype)
    for k in range(T):
        y = obs_t[k].reshape(m_dim)

        m_hat = A_t @ m
        C_hat = A_t @ C @ A_t.T + Sigma_t
        m_hat_y = H_t @ m_hat
        C_hat_vy = C_hat @ H_t.T
        C_hat_yy = H_t @ C_hat @ H_t.T + Gamma_t

        K = C_hat_vy @ torch.linalg.pinv(C_hat_yy)

        m = m_hat + K @ (y - m_hat_y)
        C = (I - K @ H_t) @ C_hat @ (I - K @ H_t).T + K @ Gamma_t @ K.T
        C = symmetrize_torch(C)

        m_hist[k] = m
        C_hist[k] = C
        K_hist[k] = K

    m_hist_np = m_hist.detach().cpu().numpy()
    C_hist_np = C_hist.detach().cpu().numpy()
    K_hist_np = K_hist.detach().cpu().numpy()
    if return_gain:
        return m_hist_np, C_hist_np, K_hist_np
    return m_hist_np, C_hist_np


def run_affine_gaussian_filter(obs, A, H, Sigma, Gamma, m0=None, C0=None, mode="kalman", return_gain=False):
    if mode != "kalman":
        raise ValueError("Section 3 currently implements only mode='kalman'.")
    if TORCH_GPU_AVAILABLE:
        return run_affine_gaussian_filter_torch(obs, A, H, Sigma, Gamma, m0=m0, C0=C0, return_gain=return_gain)

    T = len(obs)
    d = A.shape[0]
    m_dim = H.shape[0]

    if m0 is None:
        m0 = np.zeros(d)
    if C0 is None:
        C0 = 2.0 * np.eye(d)

    m = m0.astype(float)
    C = C0.astype(float)

    m_hist = np.zeros((T, d))
    C_hist = np.zeros((T, d, d))
    K_hist = np.zeros((T, d, m_dim))

    I = np.eye(d)
    for k in range(T):
        y = obs[k].reshape(m_dim)

        # Predictive moments
        m_hat = A @ m
        C_hat = A @ C @ A.T + Sigma
        m_hat_y = H @ m_hat
        C_hat_vy = C_hat @ H.T
        C_hat_yy = H @ C_hat @ H.T + Gamma

        # K = C_hat^{vy} (C_hat^{yy})^{-1}
        K = C_hat_vy @ np.linalg.pinv(C_hat_yy)

        # Analysis moments
        m = m_hat + K @ (y - m_hat_y)
        C = (I - K @ H) @ C_hat @ (I - K @ H).T + K @ Gamma @ K.T
        C = symmetrize(C)

        m_hist[k] = m
        C_hist[k] = C
        K_hist[k] = K

    if return_gain:
        return m_hist, C_hist, K_hist
    return m_hist, C_hist


kf_slider_layout = widgets.Layout(width="260px")
kf_control_row_layout = widgets.Layout(display="flex", flex_flow="row wrap", align_items="center", gap="10px")
kf_button_row_layout = widgets.Layout(display="flex", flex_flow="row", align_items="center", margin="6px 0 0 0")

kf_steps = widgets.IntSlider(value=90, min=30, max=180, step=5, description="steps", layout=kf_slider_layout)
kf_theta = widgets.FloatSlider(value=0.3, min=0.05, max=1.2, step=0.05, description="theta", layout=kf_slider_layout)
kf_sigma_v = widgets.FloatSlider(value=0.01, min=0.001, max=0.2, step=0.001, description="sigma_v", layout=kf_slider_layout)
kf_sigma_y = widgets.FloatSlider(value=1.0, min=0.1, max=3.0, step=0.1, description="sigma_y", layout=kf_slider_layout)
kf_init = widgets.Checkbox(value=True, description="Use True Init N(v0, I)")

kf_btn = widgets.Button(description="Run Linear KF", button_style="primary")
kf_out = widgets.Output()


def plot_linear_kf(_):
    kf_out.clear_output(wait=True)
    with kf_out:
        SEED = 42
        init_rng = np.random.default_rng(SEED + 10)

        truth, obs, A, H, Sigma, Gamma = simulate_linear_gaussian_2d(
            T=kf_steps.value,
            theta=kf_theta.value,
            sigma_v=kf_sigma_v.value,
            sigma_y=kf_sigma_y.value,
            seed=SEED,
        )

        if kf_init.value:
            m0 = truth[0].copy()
        else:
            m0 = truth[0] + init_rng.normal(0.0, 0.5, size=A.shape[0])
        C0 = np.eye(A.shape[0])

        m_kf, C_kf, K_kf = run_affine_gaussian_filter(
            obs,
            A,
            H,
            Sigma,
            Gamma,
            m0=m0,
            C0=C0,
            mode="kalman",
            return_gain=True,
        )

        t = np.arange(kf_steps.value)
        std_v1 = np.sqrt(np.clip(C_kf[:, 0, 0], 1e-12, None))
        err_norm = np.linalg.norm(m_kf - truth, axis=1)
        rmse_kf = np.sqrt(np.mean(np.sum((m_kf - truth) ** 2, axis=1)))

        fig, axes = plt.subplots(2, 2, figsize=(14, 9))

        ax = axes[0, 0]
        ax.plot(t, truth[:, 0], color="black", lw=2.0, label="true v1")
        ax.scatter(t, obs[:, 0], color="gray", s=10, alpha=0.5, label="obs")
        ax.plot(t, m_kf[:, 0], color="tab:blue", lw=1.8, label="KF mean")
        ax.fill_between(t, m_kf[:, 0] - 2 * std_v1, m_kf[:, 0] + 2 * std_v1, color="tab:blue", alpha=0.2)
        ax.set_title("Observed channel (v1)")
        ax.set_xlabel("time index")
        ax.set_ylabel("value")
        ax.legend(loc="upper right")

        ax = axes[0, 1]
        ax.plot(truth[:, 0], truth[:, 1], color="black", lw=2.0, label="truth")
        ax.plot(m_kf[:, 0], m_kf[:, 1], color="tab:blue", lw=1.8, label="KF")
        ax.set_title("Phase-space trajectory")
        ax.set_xlabel("v1")
        ax.set_ylabel("v2")
        ax.legend(loc="upper right")

        ax = axes[1, 0]
        ax.plot(t, err_norm, color="tab:red", lw=1.8)
        ax.set_title(f"State error norm (RMSE={rmse_kf:.3f})")
        ax.set_xlabel("time index")
        ax.set_ylabel("||m - truth||_2")

        ax = axes[1, 1]
        ax.plot(t, K_kf[:, 0, 0], color="tab:green", lw=1.8, label="K[0,0]")
        ax.plot(t, K_kf[:, 1, 0], color="tab:orange", lw=1.8, label="K[1,0]")
        ax.set_title("Kalman gain components")
        ax.set_xlabel("time index")
        ax.set_ylabel("gain")
        ax.legend(loc="upper right")

        fig.tight_layout()
        plt.show()


kf_btn.on_click(plot_linear_kf, remove=True)
kf_btn.on_click(plot_linear_kf)

display(widgets.HTML("<h4>Section 3: Linear Kalman Filter Controls</h4>"))
display(widgets.VBox([
    widgets.HBox([kf_steps, kf_theta, kf_sigma_v, kf_sigma_y], layout=kf_control_row_layout),
    widgets.HBox([kf_init, kf_btn], layout=kf_button_row_layout),
]))
display(kf_out)


## 4. Ensemble Filters

As in Section 2, at time step $j$ we approximate the filtering measure by an empirical measure with $N$ particles (ensemble members):
$$
\pi_j \approx \pi_j^N = \frac{1}{N}\sum_{n=1}^N \delta_{v_j^{(n)}}.
$$
After forecast, we similarly have
$$
\widehat\pi_{j+1}^N = \frac{1}{N}\sum_{n=1}^N \delta_{\widehat v_{j+1}^{(n)}}.
$$

The key reason to use ensemble filters is that in high dimensions we avoid propagating full covariance matrices directly. Instead, we use ensemble statistics:
$$
\bar v_{j+1}^f = \frac{1}{N}\sum_{n=1}^N \widehat v_{j+1}^{(n)}, \qquad
\bar y_{j+1}^f = \frac{1}{N}\sum_{n=1}^N h\!\left(\widehat v_{j+1}^{(n)}\right),|
$$
$$
\widehat C_{j+1}^{vy} \approx \frac{1}{N-1}\sum_{n=1}^N
\left(\widehat v_{j+1}^{(n)}-\bar v_{j+1}^f\right)
\left(h(\widehat v_{j+1}^{(n)})-\bar y_{j+1}^f\right)^T,
$$
$$
\widehat C_{j+1}^{yy} \approx \frac{1}{N-1}\sum_{n=1}^N
\left(h(\widehat v_{j+1}^{(n)})-\bar y_{j+1}^f\right)
\left(h(\widehat v_{j+1}^{(n)})-\bar y_{j+1}^f\right)^T + \Gamma.
$$
Then the ensemble Kalman gain is
$$
K_{j+1}=\widehat C_{j+1}^{vy}\left(\widehat C_{j+1}^{yy}\right)^{-1}.
$$

### 4.1 Stochastic EnKF (Perturbed Observations)

Given forecast members $\{\widehat v_{j+1}^{(n)}\}_{n=1}^N$ and observation $y_{j+1}^{\dagger}$:

1. Compute $Y_{j+1}^{(n)}=h\!\left(\widehat v_{j+1}^{(n)}\right)$ and estimate $K_{j+1}$ from ensemble covariances.
2. Draw independent perturbations
$$
\eta_{j+1}^{(n)}\sim\mathcal N(0,\Gamma),\quad n=1,\dots,N.
$$
3. Update each ensemble member with a perturbed innovation:
$$
v_{j+1}^{(n),a}=\widehat v_{j+1}^{(n)}+K_{j+1}\Big(y_{j+1}^{\dagger}+\eta_{j+1}^{(n)}-Y_{j+1}^{(n)}\Big).
$$

This stochastic update gives the correct analysis covariance in expectation, but introduces additional Monte Carlo noise through $\eta_{j+1}^{(n)}$.

### 4.2 EnSRF (Deterministic Square-Root Analysis)

EnSRF removes the observation perturbation and updates mean/anomalies deterministically.

Let forecast mean and anomalies be
$$
\bar v_{j+1}^f=\frac{1}{N}\sum_{n=1}^N\widehat v_{j+1}^{(n)},\qquad
a_{j+1}^{(n),f}=\widehat v_{j+1}^{(n)}-\bar v_{j+1}^f.
$$
For the serial (one-observation-at-a-time) form used in code, for each observed component $\ell$:

1. Forecast observation anomalies:
$$
d_{\ell}^{(n)} = h_{\ell}\!\left(\widehat v_{j+1}^{(n)}\right)-\bar y_{j+1,\ell}^f.
$$
2. Scalar covariances:
$$
p_{yy,\ell}=\frac{1}{N-1}\sum_{n=1}^N (d_{\ell}^{(n)})^2 + \gamma_{\ell},
\qquad
p_{xy,\ell}=\frac{1}{N-1}\sum_{n=1}^N a_{j+1}^{(n),f}d_{\ell}^{(n)}.
$$
3. Mean update:
$$
K_{\ell}=\frac{p_{xy,\ell}}{p_{yy,\ell}},\qquad
\bar v_{j+1}^a \leftarrow \bar v_{j+1}^a + K_{\ell}\big(y_{j+1,\ell}^{\dagger}-\bar y_{j+1,\ell}^f\big).
$$
4. Anomaly square-root update:
$$
\alpha_{\ell}=\frac{1}{1+\sqrt{\gamma_{\ell}/p_{yy,\ell}}},
\qquad
\widetilde K_{\ell}=\alpha_{\ell}K_{\ell},
$$
$$
a_{j+1}^{(n),a} \leftarrow a_{j+1}^{(n),f} - \widetilde K_{\ell} d_{\ell}^{(n)}.
$$
5. Recompose members: $v_{j+1}^{(n),a}=\bar v_{j+1}^a+a_{j+1}^{(n),a}$.

Compared with particle filters, ensemble members remain **equally weighted** throughout. There is no importance-weight degeneracy/resampling loop; instead, the update is driven by global ensemble properties (means, covariances, anomalies).

Below we implement two ensemble filters on the same two testbeds (Linear-Gaussian and Lorenz-63):
- Stochastic EnKF
- Ensemble Square-Root Filter (EnSRF)

Both examples include a **post-analysis inflation** factor $r_{\text{infl}}$ (default $1.0$, i.e. no inflation):
$$
v_{j+1}^{(n),a} \leftarrow \bar v_{j+1}^{a} + r_{\text{infl}}\Big(v_{j+1}^{(n),a}-\bar v_{j+1}^{a}\Big),\qquad n=1,\dots,N.
$$


In [ ]:
from IPython.display import display


def analysis_enkf_stochastic(Xf, y_obs, h_func, Gamma, rng):
    N = Xf.shape[0]
    Yf = np.array([h_func(x) for x in Xf])

    xbar = Xf.mean(axis=0)
    ybar = Yf.mean(axis=0)

    Xp = Xf - xbar
    Yp = Yf - ybar

    Pxy = (Xp.T @ Yp) / max(N - 1, 1)
    Pyy = (Yp.T @ Yp) / max(N - 1, 1) + Gamma

    K = Pxy @ np.linalg.pinv(Pyy)
    pert = rng.multivariate_normal(np.zeros(len(y_obs)), Gamma, size=N)

    Xa = Xf + (y_obs + pert - Yf) @ K.T
    return Xa


def analysis_ensrf(Xf, y_obs, h_func, gamma_diag):
    # Serial square-root update for (approximately) diagonal observation error covariance.
    X = Xf.copy()
    m_obs = len(y_obs)

    for ell in range(m_obs):
        Y = np.array([h_func(x) for x in X])
        y_f = Y[:, ell]

        x_mean = X.mean(axis=0)
        A = X - x_mean
        y_mean = np.mean(y_f)
        dy = y_f - y_mean

        r = max(float(gamma_diag[ell]), 1e-12)
        p_yy = (dy @ dy) / max(len(dy) - 1, 1) + r
        p_xy = (A.T @ dy) / max(len(dy) - 1, 1)

        K = p_xy / p_yy
        x_mean = x_mean + K * (y_obs[ell] - y_mean)

        alpha = 1.0 / (1.0 + np.sqrt(r / p_yy))
        K_tilde = alpha * K
        A = A - np.outer(dy, K_tilde)

        X = x_mean + A

    return X


def post_analysis_inflate(Xa, infl_rate):
    if np.isclose(infl_rate, 1.0):
        return Xa
    xbar = Xa.mean(axis=0, keepdims=True)
    return xbar + infl_rate * (Xa - xbar)


def analysis_enkf_stochastic_torch(Xf, y_obs, H, Gamma):
    N = Xf.shape[0]
    Yf = Xf @ H.T

    xbar = Xf.mean(dim=0)
    ybar = Yf.mean(dim=0)

    Xp = Xf - xbar
    Yp = Yf - ybar

    Pxy = (Xp.T @ Yp) / max(N - 1, 1)
    Pyy = (Yp.T @ Yp) / max(N - 1, 1) + Gamma

    K = Pxy @ torch.linalg.pinv(Pyy)
    pert = mvn_sample_torch(
        torch.zeros(y_obs.shape[0], device=Xf.device, dtype=Xf.dtype),
        Gamma,
        size=N,
    )
    Xa = Xf + (y_obs.unsqueeze(0) + pert - Yf) @ K.T
    return Xa


def analysis_ensrf_torch(Xf, y_obs, H, gamma_diag):
    X = Xf.clone()
    m_obs = y_obs.shape[0]

    for ell in range(m_obs):
        y_f = X @ H[ell]

        x_mean = X.mean(dim=0)
        A = X - x_mean
        y_mean = torch.mean(y_f)
        dy = y_f - y_mean

        r = torch.clamp(gamma_diag[ell], min=1e-12)
        p_yy = (dy @ dy) / max(len(dy) - 1, 1) + r
        p_xy = (A.T @ dy) / max(len(dy) - 1, 1)

        K = p_xy / p_yy
        x_mean = x_mean + K * (y_obs[ell] - y_mean)

        alpha = 1.0 / (1.0 + torch.sqrt(r / p_yy))
        K_tilde = alpha * K
        A = A - torch.outer(dy, K_tilde)
        X = x_mean + A

    return X


def post_analysis_inflate_torch(Xa, infl_rate):
    if np.isclose(infl_rate, 1.0):
        return Xa
    xbar = Xa.mean(dim=0, keepdim=True)
    return xbar + infl_rate * (Xa - xbar)


def run_ensemble_linear_torch(
    obs,
    A,
    H,
    Sigma,
    Gamma,
    N,
    method,
    use_true_init,
    truth_x0,
    inflation,
    seed,
    progress_bar=None,
):
    device = torch.device("cuda")
    torch.manual_seed(int(seed))
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(int(seed))

    obs_t = as_torch(obs, device=device, dtype=TORCH_DTYPE)
    A_t = as_torch(A, device=device, dtype=TORCH_DTYPE)
    H_t = as_torch(H, device=device, dtype=TORCH_DTYPE)
    Sigma_t = as_torch(Sigma, device=device, dtype=TORCH_DTYPE)
    Gamma_t = as_torch(Gamma, device=device, dtype=TORCH_DTYPE)
    truth_x0_t = as_torch(truth_x0, device=device, dtype=TORCH_DTYPE)

    T = obs_t.shape[0]
    d = A_t.shape[0]

    if use_true_init:
        X = mvn_sample_torch(truth_x0_t, torch.eye(d, device=device, dtype=TORCH_DTYPE), size=N)
    else:
        offset = torch.normal(mean=0.0, std=0.5, size=(d,), device=device, dtype=TORCH_DTYPE)
        X = mvn_sample_torch(truth_x0_t + offset, torch.eye(d, device=device, dtype=TORCH_DTYPE), size=N)

    est = torch.zeros((T, d), device=device, dtype=TORCH_DTYPE)
    zero_d = torch.zeros(d, device=device, dtype=TORCH_DTYPE)

    for k in range(T):
        X = X @ A_t.T + mvn_sample_torch(zero_d, Sigma_t, size=N)

        y = obs_t[k].reshape(-1)
        if method == "enkf":
            X = analysis_enkf_stochastic_torch(X, y, H_t, Gamma_t)
        elif method == "ensrf":
            X = analysis_ensrf_torch(X, y, H_t, torch.diag(Gamma_t))
        else:
            raise ValueError(f"Unknown ensemble method: {method}")

        X = post_analysis_inflate_torch(X, inflation)
        est[k] = X.mean(dim=0)

        if progress_bar:
            progress_bar.value += 1

    return est.detach().cpu().numpy()


def run_ensemble_lorenz_torch(
    obs,
    obs_times,
    obs_idx,
    dt,
    sigma_v,
    sigma_y,
    N,
    method,
    use_true_init,
    truth_x0,
    inflation,
    seed,
    progress_bar=None,
):
    device = torch.device("cuda")
    torch.manual_seed(int(seed))
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(int(seed))

    T_obs = len(obs)
    d = 3

    H_t = torch.zeros((len(obs_idx), d), device=device, dtype=TORCH_DTYPE)
    for r, c in enumerate(obs_idx):
        H_t[r, c] = 1.0

    Gamma_t = (sigma_y ** 2) * torch.eye(len(obs_idx), device=device, dtype=TORCH_DTYPE)
    truth_x0_t = as_torch(truth_x0, device=device, dtype=TORCH_DTYPE)

    if use_true_init:
        X = mvn_sample_torch(truth_x0_t, torch.eye(d, device=device, dtype=TORCH_DTYPE), size=N)
    else:
        offset = torch.normal(mean=0.0, std=2.0, size=(d,), device=device, dtype=TORCH_DTYPE)
        X = mvn_sample_torch(truth_x0_t + offset, torch.eye(d, device=device, dtype=TORCH_DTYPE), size=N)

    obs_t = as_torch(obs, device=device, dtype=TORCH_DTYPE)
    est = torch.zeros((T_obs, d), device=device, dtype=TORCH_DTYPE)
    current_time_step = 0

    for k in range(T_obs):
        target_time_step = int(obs_times[k])
        steps_to_sim = max(target_time_step - current_time_step, 0)

        for _ in range(steps_to_sim):
            X = lorenz63_step_torch(X, dt)
            if sigma_v > 0:
                X = X + sigma_v * torch.randn_like(X)

        y = obs_t[k].reshape(-1)
        if method == "enkf":
            X = analysis_enkf_stochastic_torch(X, y, H_t, Gamma_t)
        elif method == "ensrf":
            X = analysis_ensrf_torch(X, y, H_t, torch.diag(Gamma_t))
        else:
            raise ValueError(f"Unknown ensemble method: {method}")

        X = post_analysis_inflate_torch(X, inflation)
        est[k] = X.mean(dim=0)
        current_time_step = target_time_step

        if progress_bar:
            progress_bar.value += 1

    return est.detach().cpu().numpy()


def run_ensemble_linear(
    obs,
    A,
    H,
    Sigma,
    Gamma,
    N,
    method,
    use_true_init,
    truth_x0,
    inflation,
    seed,
    progress_bar=None,
):
    if TORCH_GPU_AVAILABLE:
        return run_ensemble_linear_torch(
            obs,
            A,
            H,
            Sigma,
            Gamma,
            N,
            method,
            use_true_init,
            truth_x0,
            inflation,
            seed,
            progress_bar=progress_bar,
        )

    rng = np.random.default_rng(seed)
    T = len(obs)
    d = A.shape[0]

    if use_true_init:
        X = rng.multivariate_normal(truth_x0, np.eye(d), size=N)
    else:
        offset = rng.normal(0.0, 0.5, size=d)
        X = rng.multivariate_normal(truth_x0 + offset, np.eye(d), size=N)

    est = np.zeros((T, d))

    def h_lin(x):
        return (H @ x).reshape(-1)

    for k in range(T):
        # Forecast
        X = X @ A.T + rng.multivariate_normal(np.zeros(d), Sigma, size=N)

        # Analysis
        y = obs[k].reshape(-1)
        if method == "enkf":
            X = analysis_enkf_stochastic(X, y, h_lin, Gamma, rng)
        elif method == "ensrf":
            X = analysis_ensrf(X, y, h_lin, np.diag(Gamma))
        else:
            raise ValueError(f"Unknown ensemble method: {method}")

        # Post-analysis inflation
        X = post_analysis_inflate(X, inflation)

        est[k] = X.mean(axis=0)

        if progress_bar:
            progress_bar.value += 1

    return est


def run_ensemble_lorenz(
    obs,
    obs_times,
    obs_idx,
    dt,
    sigma_v,
    sigma_y,
    N,
    method,
    use_true_init,
    truth_x0,
    inflation,
    seed,
    progress_bar=None,
):
    if TORCH_GPU_AVAILABLE:
        return run_ensemble_lorenz_torch(
            obs,
            obs_times,
            obs_idx,
            dt,
            sigma_v,
            sigma_y,
            N,
            method,
            use_true_init,
            truth_x0,
            inflation,
            seed,
            progress_bar=progress_bar,
        )

    rng = np.random.default_rng(seed)
    T_obs = len(obs)
    d = 3

    H = np.zeros((len(obs_idx), d))
    for r, c in enumerate(obs_idx):
        H[r, c] = 1.0

    Gamma = (sigma_y ** 2) * np.eye(len(obs_idx))

    if use_true_init:
        X = rng.multivariate_normal(truth_x0, np.eye(d), size=N)
    else:
        offset = rng.normal(0.0, 2.0, size=d)
        X = rng.multivariate_normal(truth_x0 + offset, np.eye(d), size=N)

    est = np.zeros((T_obs, d))
    current_time_step = 0

    def h_l63(x):
        return x[obs_idx]

    for k in range(T_obs):
        target_time_step = obs_times[k]
        steps_to_sim = target_time_step - current_time_step

        # Forecast
        for i in range(N):
            temp = X[i]
            for _ in range(steps_to_sim):
                temp = lorenz63_step(temp, dt)
                if sigma_v > 0:
                    temp += rng.normal(0.0, sigma_v, size=d)
            X[i] = temp

        # Analysis
        y = np.atleast_1d(obs[k])
        if method == "enkf":
            X = analysis_enkf_stochastic(X, y, h_l63, Gamma, rng)
        elif method == "ensrf":
            X = analysis_ensrf(X, y, h_l63, np.diag(Gamma))
        else:
            raise ValueError(f"Unknown ensemble method: {method}")

        # Post-analysis inflation
        X = post_analysis_inflate(X, inflation)

        est[k] = X.mean(axis=0)
        current_time_step = target_time_step

        if progress_bar:
            progress_bar.value += 1

    return est


# -------------------------------------------------------------------------
# UI & Plotting Logic (Section 4, aligned with Section 2 style)
# -------------------------------------------------------------------------

style_layout = widgets.Layout(width="260px")
btn_layout = widgets.Layout(width="120px")
progress_layout = widgets.Layout(width="280px", margin="5px 0 0 10px")

# --- Linear widgets ---
ens_lg_particles = widgets.IntSlider(value=20, min=5, max=100, step=5, description="Ens size (N)", layout=style_layout)
ens_lg_init = widgets.Checkbox(value=True, description="Use True Init N(v0, I)")
ens_lg_infl = widgets.FloatSlider(value=1.0, min=1.0, max=1.30, step=0.01, description="Infl rate", layout=style_layout)
ens_lg_btn = widgets.Button(description="Run Linear", button_style="info", icon="play", layout=btn_layout)
ens_lg_progress = widgets.IntProgress(value=0, min=0, max=100, description="Progress:", bar_style="info", layout=progress_layout)
ens_lg_out = widgets.Output()

# --- Lorenz widgets ---
ens_l63_particles = widgets.IntSlider(value=20, min=5, max=100, step=5, description="Ens size (N)", layout=style_layout)
ens_l63_init = widgets.Checkbox(value=True, description="Use True Init N(v0, I)")
ens_l63_infl = widgets.FloatSlider(value=1.0, min=1.0, max=1.30, step=0.01, description="Infl rate", layout=style_layout)
ens_l63_btn = widgets.Button(description="Run Lorenz", button_style="info", icon="play", layout=btn_layout)
ens_l63_progress = widgets.IntProgress(value=0, min=0, max=100, description="Progress:", bar_style="info", layout=progress_layout)
ens_l63_out = widgets.Output()


def plot_ensemble_linear_callback(_):
    ens_lg_out.clear_output(wait=True)

    # Simulation params
    T_sim = 80
    theta = 0.3
    sv = 0.01
    sy = 1.0
    SEED = 42

    # Ensemble params
    N = ens_lg_particles.value
    use_init = ens_lg_init.value
    inflation = ens_lg_infl.value

    methods = [
        ("Stochastic EnKF", "enkf", "tab:green"),
        ("EnSRF", "ensrf", "tab:orange"),
    ]

    ens_lg_progress.value = 0
    ens_lg_progress.max = len(methods) * T_sim
    ens_lg_progress.description = "Running..."

    with ens_lg_out:
        truth, obs, A, H, Sigma, Gamma = simulate_linear_gaussian_2d(
            T=T_sim,
            theta=theta,
            sigma_v=sv,
            sigma_y=sy,
            seed=SEED,
        )

        results = []
        rmses = []

        for name, method_code, color in methods:
            est = run_ensemble_linear(
                obs,
                A,
                H,
                Sigma,
                Gamma,
                N,
                method_code,
                use_init,
                truth[0],
                inflation,
                seed=SEED + 10,
                progress_bar=ens_lg_progress,
            )
            rmse = np.sqrt(np.mean(np.sum((est - truth) ** 2, axis=1)))
            results.append((name, est, color))
            rmses.append(rmse)

        ens_lg_progress.description = "Done"

        fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

        ax = axes[0]
        ax.plot(truth[:, 0], truth[:, 1], "k-", lw=2, label="Truth")
        for name, est, col in results:
            ax.plot(est[:, 0], est[:, 1], color=col, lw=1.3, alpha=0.9, label=name)
        ax.set_title("2D Phase Trajectory")
        ax.legend()

        ax = axes[1]
        t = np.arange(T_sim)
        ax.plot(t, truth[:, 0], "k-", label="Truth (Obs Dim)")
        ax.scatter(t, obs[:, 0], c="gray", s=10, alpha=0.5, label="Obs")
        for name, est, col in results:
            ax.plot(t, est[:, 0], color=col, lw=1.3, alpha=0.9)
        ax.set_title("Observed Dimension vs Time")

        ax = axes[2]
        bars = ax.bar([m[0] for m in methods], rmses, color=[m[2] for m in methods])
        ax.set_title(f"RMSE (infl={inflation:.2f})")
        ax.set_ylabel("RMSE")
        for bar in bars:
            h = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2.0, h, f"{h:.3f}", ha="center", va="bottom")

        plt.tight_layout()
        plt.show()


def plot_ensemble_lorenz_callback(_):
    ens_l63_out.clear_output(wait=True)

    # Simulation params
    T_sim = 30.0
    dt = 0.03
    Delta_t = 0.15
    sv = 0.01
    sy = 1.0
    SEED = 99

    # Ensemble params
    N = ens_l63_particles.value
    use_init = ens_l63_init.value
    inflation = ens_l63_infl.value

    methods = [
        ("Stochastic EnKF", "enkf", "tab:green"),
        ("EnSRF", "ensrf", "tab:orange"),
    ]

    with ens_l63_out:
        truth_full, obs, obs_times, obs_idx = simulate_lorenz63(
            T=T_sim,
            dt=dt,
            Delta_t=Delta_t,
            sigma_v=sv,
            sigma_y=sy,
            burn_steps=100,
            seed=SEED,
        )

        truth_at_obs = truth_full[obs_times]

        ens_l63_progress.value = 0
        ens_l63_progress.max = len(methods) * len(obs)
        ens_l63_progress.description = "Running..."

        results = []
        rmses = []

        for name, method_code, color in methods:
            est = run_ensemble_lorenz(
                obs,
                obs_times,
                obs_idx,
                dt,
                sv,
                sy,
                N,
                method_code,
                use_init,
                truth_full[0],
                inflation,
                seed=SEED + 50,
                progress_bar=ens_l63_progress,
            )

            rmse = np.sqrt(np.mean(np.sum((est - truth_at_obs) ** 2, axis=1)))
            results.append((name, est, color))
            rmses.append(rmse)

        ens_l63_progress.description = "Done"

        fig = plt.figure(figsize=(16, 5))

        ax = fig.add_subplot(1, 3, 1, projection="3d")
        ax.plot(
            truth_full[:, 0],
            truth_full[:, 1],
            truth_full[:, 2],
            "k-",
            lw=0.5,
            alpha=0.3,
            label="Full Truth",
        )
        for name, est, col in results:
            ax.plot(est[:, 0], est[:, 1], est[:, 2], color=col, lw=1.5, label=name)
        ax.set_title("3D Trajectory (Estimates)")
        ax.legend()

        ax = fig.add_subplot(1, 3, 2)
        t_full = np.arange(len(truth_full)) * dt
        t_obs = obs_times * dt
        ax.plot(t_full, truth_full[:, 0], "k-", lw=1, alpha=0.5, label="Truth X")
        ax.scatter(t_obs, obs[:, 0], c="gray", s=10, alpha=0.5, label="Obs")
        for name, est, col in results:
            ax.plot(t_obs, est[:, 0], color=col, marker="o", markersize=3, lw=1, label=name)
        ax.set_title("Observed Dimension X vs Time")
        ax.legend()

        ax = fig.add_subplot(1, 3, 3)
        bars = ax.bar([m[0] for m in methods], rmses, color=[m[2] for m in methods])
        ax.set_title(f"RMSE (infl={inflation:.2f})")
        ax.set_ylabel("RMSE")
        for bar in bars:
            h = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2.0, h, f"{h:.3f}", ha="center", va="bottom")

        plt.tight_layout()
        plt.show()


ens_lg_btn.on_click(plot_ensemble_linear_callback, remove=True)
ens_lg_btn.on_click(plot_ensemble_linear_callback)

ens_l63_btn.on_click(plot_ensemble_lorenz_callback, remove=True)
ens_l63_btn.on_click(plot_ensemble_lorenz_callback)


display(widgets.HTML("<h3>Section 4 - System 1: Linear Gaussian (2D)</h3>"))
display(widgets.HBox([ens_lg_particles, ens_lg_init, ens_lg_infl, ens_lg_btn, ens_lg_progress]))
display(ens_lg_out)

display(widgets.HTML("<hr><h3>Section 4 - System 2: Lorenz-63 (Chaotic)</h3>"))
display(widgets.HBox([ens_l63_particles, ens_l63_init, ens_l63_infl, ens_l63_btn, ens_l63_progress]))
display(ens_l63_out)
